In [2]:
# FUNCTIONS IN MODULE
from modules.psu_pg_functions import *
import pandas as pd

import time

# VARIABLES
posVoltage = 40
negVoltage = 40

ModuleNotFoundError: No module named 'modules'

In [ ]:
# START SERIAL CONNECTION AND SAVE THE CONNECTIONS IN A LIST: [PSU, PG]
serials_list = serial_start_connections()
serials_list    # SHOW SERIAL LIST IN JUPYTER

In [ ]:
# SEND PSU SETPOINTS (45, -45 VOLTS) AND READ THE RESPONSE AFTER 5 SECONDS
status = send_PSU_setpoints(serials_list[0], posVoltage, negVoltage, 1)
print(status)

time.sleep(5)
for i in range(5):
    data, crc = read_PSU_data(serials_list[0])
    print(data)
    print(crc)
    print("")

In [ ]:
# ENABLE THE PSU
status = send_PSU_enable(serials_list[0], 1)
print(status)

In [ ]:
# SEND TIMES TO THE PG
status = send_PG_pulsetimes(serials_list[1], repRate=5000, pulseLength=75, onTime=248, verbose=1)
print(status)

In [ ]:
# ENABLE THE PG AND GET THE ZERO-DATA
zero_data = send_PG_enable(serials_list[1], 1)
print(zero_data)

In [ ]:
# READ THE NEXT PULSE (ONE SINGLE PULSE)
pulse_array, pulse_datalength = read_next_PG_pulse(serials_list[1], verbose=1)
print(pulse_datalength)
pulse_array.shape

In [ ]:
# CONVERT THE PULSE TO A DATA FRAME
df_pulse = pd.DataFrame(pulse_array, columns=["Voltage", "Current"])
df_pulse

In [ ]:
# ADD THE OFFSET (ZEROVOLTAGE, ZEROCURRENT) TO THE DF
df_pulse_zero = df_pulse.copy()
df_pulse_zero["Voltage"] = df_pulse["Voltage"] - zero_data[0]
df_pulse_zero["Current"] = df_pulse["Current"] - zero_data[1]
print(df_pulse_zero)

In [ ]:
# PLOT THE PULSE
df_pulse_zero.plot(
    title="PG received data: voltage, current",
    y=['Voltage', 'Current'],
    figsize=(15,5),
    color=['red', 'blue'],
    style=['-', '-'],
    grid=True,
    )
plt.xlabel('samples')
#plt.xlabel('time [ms]')
plt.ylabel('value')


plt.show()

In [ ]:
status = send_PG_disable(serials_list[1], 1)
print(status)

In [ ]:
status = send_PSU_disable(serials_list[0], 1)
print(status)

In [ ]:
status = serial_close_connections(serials_list, 1)
print(status)